# Data pre-processing notebook
### Dropping non-songs, encoding categorical variables

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/tracks_drop_duplicates.csv")
df.head()

,Unnamed: 0,index,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0,0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,...,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,1,1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,...,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,2,2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,...,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,3,3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,...,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,4,4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,...,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


Data cleanup

In [2]:
!pip install category_encoders

In [6]:
import category_encoders as ce

def clean_data(df):
    # Drop the id column, name, release_date
    x = df.copy()
    drop_cols = ['Unnamed: 0', 'index', 'id', 'name', 'release_date', 'id_artists', 'artists']
    x = x.drop(labels=drop_cols, axis=1)
    
    # Drop zero tempo songs
    x = x[x['tempo']!=0]
    
    # Do one-hot encoding for key
    ohe = ce.one_hot.OneHotEncoder(cols=['key'])
    x = ohe.fit_transform(x)
    
    
    # Normalize numeric features
    num_features = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature']
    
    for i in num_features:
        col_mean = x[i].values.mean()
        col_var = np.var(df[i])
        col_norm = [(o - col_mean) / col_var**(1/2) for o in x[i].values]
        x[i] = col_norm
    
    return x


In [7]:

clean_df = clean_data(df)
print(f'Old df shape: {df.shape}\nOld df columns: {[i for i in df.columns]}\n\n'
        f'Cleaned df shape: {clean_df.shape}\nClean df columns: {[i for i in clean_df.columns]}')

/Users/Carl/miniconda3/envs/U4-S2-NN/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Old df shape: (446475, 22)
Old df columns: ['Unnamed: 0', 'index', 'id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists', 'id_artists', 'release_date', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

Cleaned df shape: (446198, 26)
Clean df columns: ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_12', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']


In [8]:
clean_df.describe()

,popularity,duration_ms,explicit,danceability,energy,key_1,key_2,key_3,key_4,key_5,...,key_12,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,4.461980e+05,4.461980e+05,4.461980e+05,4.461980e+05,4.461980e+05,446198.000000,446198.000000,446198.000000,446198.000000,446198.000000,...,446198.000000,4.461980e+05,446198.000000,4.461980e+05,4.461980e+05,4.461980e+05,4.461980e+05,4.461980e+05,4.461980e+05,4.461980e+05
mean,-2.598859e-17,-7.732880e-17,4.204037e-17,9.629792e-16,-1.910926e-16,0.126845,0.072347,0.126238,0.035659,0.089841,...,0.111926,3.725031e-16,0.657195,-4.114860e-17,3.019263e-16,4.700878e-17,2.688036e-17,-3.765798e-16,-1.133561e-15,-8.917654e-18
std,9.997841e-01,9.945503e-01,1.000250e+00,9.967265e-01,9.993776e-01,0.332800,0.259062,0.332117,0.185439,0.285955,...,0.315275,9.971516e-01,0.474648,1.000205e+00,9.998721e-01,9.990741e-01,9.996037e-01,9.988576e-01,9.954012e-01,9.808388e-01
min,-1.480897e+00,-1.595024e+00,-2.189212e-01,-3.077494e+00,-2.156647e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.715866e+00,0.000000,-5.857257e-01,-1.295942e+00,-4.227010e-01,-1.127532e+00,-2.164436e+00,-2.950973e+00,-7.885510e+00
25%,-8.253583e-01,-4.241557e-01,-2.189212e-01,-6.644670e-01,-7.872860e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-5.246672e-01,0.000000,-4.137112e-01,-1.011593e+00,-4.227010e-01,-6.356965e-01,-7.950903e-01,-7.694321e-01,2.658238e-01
50%,-5.934939e-03,-1.168425e-01,-2.189212e-01,8.622575e-02,2.639235e-02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.908149e-01,1.000000,-3.582066e-01,-7.074667e-02,-4.226210e-01,-4.089712e-01,5.297011e-02,-3.431833e-02,2.658238e-01
75%,7.042320e-01,2.569606e-01,-2.189212e-01,7.348243e-01,8.162558e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7.294736e-01,1.000000,-1.711909e-01,9.564659e-01,-3.886925e-01,3.555674e-01,8.387876e-01,6.010337e-01,2.658238e-01
max,3.981925e+00,4.008877e+01,4.570127e+00,2.554504e+00,1.812516e+00,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,3.054693e+00,1.000000,4.269682e+00,1.561890e+00,3.335153e+00,4.114989e+00,1.725750e+00,4.194066e+00,2.303657e+00


In [8]:
clean_df.to_csv('data/songs_cleaned.csv')

FileNotFoundError: [Errno 2] No such file or directory: './Data/songs_cleaned.csv'